In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras import layers,Model,activations,metrics,preprocessing
import lightgbm as lgb
from plotly import graph_objects as go

In [63]:
train_identity=pd.read_csv('data/train_identity.csv',index_col=0)
train_transaction=pd.read_csv('data/train_transaction.csv',index_col=0)

In [64]:
train_identity.iloc[0]

id_01                                     0
id_02                                 70787
id_03                                   NaN
id_04                                   NaN
id_05                                   NaN
id_06                                   NaN
id_07                                   NaN
id_08                                   NaN
id_09                                   NaN
id_10                                   NaN
id_11                                   100
id_12                              NotFound
id_13                                   NaN
id_14                                  -480
id_15                                   New
id_16                              NotFound
id_17                                   166
id_18                                   NaN
id_19                                   542
id_20                                   144
id_21                                   NaN
id_22                                   NaN
id_23                           

In [65]:
train_transaction.iloc[100]

isFraud                     0
TransactionDT           88169
TransactionAmt            100
ProductCD                   H
card1                   16659
card2                     170
card3                     150
card4                    visa
card5                     226
card6                  credit
addr1                     330
addr2                      87
dist1                     NaN
dist2                     NaN
P_emaildomain     comcast.net
R_emaildomain             NaN
C1                          1
C2                          1
C3                          0
C4                          0
C5                          0
C6                          1
C7                          0
C8                          1
C9                          0
C10                         1
C11                         1
C12                         0
C13                         0
C14                         0
                     ...     
V310                        0
V311                        0
V312      

In [66]:
train_identity.describe(include='all')

,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
count,144233.000000,140872.000000,66324.000000,66324.000000,136865.000000,136865.000000,5155.000000,5155.000000,74926.000000,74926.000000,...,140282,77586.000000,73289,77805,140985,140985,140985,140985,140810,118666
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,130,NaN,260,4,2,2,2,2,2,1786
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,chrome 63.0,NaN,1920x1080,match_status:2,T,F,T,F,desktop,Windows
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22000,NaN,16874,60011,77814,134066,110452,73922,85165,47722
mean,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.698710,13.285354,-38.600388,0.091023,-0.301124,...,NaN,26.508597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,14.347949,159651.816856,0.598231,0.701015,5.249856,16.491104,11.384207,26.084899,0.983842,2.789446,...,NaN,3.737502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,-100.000000,1.000000,-13.000000,-28.000000,-72.000000,-100.000000,-46.000000,-100.000000,-36.000000,-100.000000,...,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,-10.000000,67992.000000,0.000000,0.000000,0.000000,-6.000000,5.000000,-48.000000,0.000000,0.000000,...,NaN,24.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,-5.000000,125800.500000,0.000000,0.000000,0.000000,0.000000,14.000000,-34.000000,0.000000,0.000000,...,NaN,24.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,-5.000000,228749.000000,0.000000,0.000000,1.000000,0.000000,22.000000,-23.000000,0.000000,0.000000,...,NaN,32.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
train_transaction.describe(include='all')

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,590540.000000,5.905400e+05,590540.000000,590540,590540.000000,581607.000000,588975.000000,588963,586281.000000,588969,...,82351.000000,82351.000000,82351.000000,82351.000000,82351.000000,82351.00000,82351.000000,82351.000000,82351.000000,82351.000000
unique,NaN,NaN,NaN,5,NaN,NaN,NaN,4,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,W,NaN,NaN,NaN,visa,NaN,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,439670,NaN,NaN,NaN,384767,NaN,439938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,0.034990,7.372311e+06,135.027176,NaN,9898.734658,362.555488,153.194925,NaN,199.278897,NaN,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
std,0.183755,4.617224e+06,239.162522,NaN,4901.170153,157.793246,11.336444,NaN,41.244453,NaN,...,4.727971,6217.223583,11169.275702,7955.735482,243.861391,387.62948,274.576920,668.486833,1095.034387,814.946722
min,0.000000,8.640000e+04,0.251000,NaN,1000.000000,100.000000,100.000000,NaN,100.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,3.027058e+06,43.321000,NaN,6019.000000,214.000000,150.000000,NaN,166.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,7.306528e+06,68.769000,NaN,9678.000000,361.000000,150.000000,NaN,226.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.124662e+07,125.000000,NaN,14184.000000,512.000000,150.000000,NaN,226.000000,NaN,...,0.000000,0.000000,25.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [7]:
train_identity.columns

Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14',
       'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22',
       'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object')

In [37]:
train_transaction.columns[50:]

Index(['M5', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=344)

In [9]:
train_transaction.loc[:,'TransactionID'].unique().size

590540

In [10]:
train_identity.shape

(144233, 41)

In [11]:
train_identity.loc[:,'TransactionID'].unique().size

144233

In [55]:
rdomain=train_transaction.loc[:,'R_emaildomain']

In [13]:
rdomain.isna().sum()

453249

In [14]:
rdomain.size

590540

In [45]:
train_transaction.loc[:,'V1'].isna().sum()

279287

In [30]:
train_identity.loc[:,'id_30'].unique()

array(['Android 7.0', 'iOS 11.1.2', nan, 'Mac OS X 10_11_6', 'Windows 10',
       'Android', 'Linux', 'iOS 11.0.3', 'Mac OS X 10_7_5',
       'Mac OS X 10_12_6', 'Mac OS X 10_13_1', 'iOS 11.1.0',
       'Mac OS X 10_9_5', 'Windows 7', 'Windows 8.1', 'Mac', 'iOS 10.3.3',
       'Mac OS X 10.12', 'Mac OS X 10_10_5', 'Mac OS X 10_11_5',
       'iOS 9.3.5', 'Android 5.1.1', 'Android 7.1.1', 'Android 6.0',
       'iOS 10.3.1', 'Mac OS X 10.9', 'iOS 11.1.1', 'Windows Vista',
       'iOS 10.3.2', 'iOS 11.0.2', 'Mac OS X 10.11', 'Android 8.0.0',
       'iOS 10.2.0', 'iOS 10.2.1', 'iOS 11.0.0', 'Mac OS X 10.10',
       'Mac OS X 10_12_3', 'Mac OS X 10_12', 'Android 6.0.1', 'iOS',
       'Mac OS X 10.13', 'Mac OS X 10_12_5', 'Mac OS X 10_8_5',
       'iOS 11.0.1', 'iOS 10.0.2', 'Android 5.0.2', 'Windows XP',
       'iOS 11.2.0', 'Mac OS X 10.6', 'Windows 8', 'Mac OS X 10_6_8',
       'Mac OS X 10_11_4', 'Mac OS X 10_12_1', 'iOS 10.1.1',
       'Mac OS X 10_11_3', 'Mac OS X 10_12_4', 'Mac OS X 10

In [46]:
nan_count=[train_transaction.loc[:,'V{}'.format(i)].isna().sum() for i in range(1,340)]

In [50]:
nan_count

[279287,
 279287,
 279287,
 279287,
 279287,
 279287,
 279287,
 279287,
 279287,
 279287,
 279287,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 76073,
 168969,
 168969,
 168969,
 168969,
 168969,
 168969,
 168969,
 168969,
 168969,
 168969,
 168969,
 168969,
 168969,
 168969,
 168969,
 168969,
 168969,
 168969,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 77096,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 89164,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 314,
 31

In [61]:
(train_transaction.loc[:,'V310']==0.0).sum()

354815

In [65]:
train_transaction.columns[:60]

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
       'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8',
       'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4',
       'M5', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5'],
      dtype='object')

In [82]:
train_transaction.loc[:,'M9'].unique()

array([nan, 'F', 'T'], dtype=object)

In [5]:
id_nan_count=[train_identity.loc[:,'id_{}'.format(i)].isna().sum() for i in range(11,39)]

In [6]:
id_nan_count

[3255,
 0,
 16913,
 64189,
 3248,
 14893,
 4864,
 99120,
 4915,
 4972,
 139074,
 139064,
 139064,
 139486,
 139101,
 139070,
 139064,
 3255,
 3255,
 66668,
 3951,
 66647,
 70944,
 66428,
 3248,
 3248,
 3248,
 3248]

In [7]:
train_transaction.loc[:,'isFraud'].sum()

20663

In [10]:
train_identity.loc[:,'DeviceType'].isna().sum()

3423

In [14]:
devices=train_identity.loc[:,'DeviceInfo'].unique()

In [17]:
devices[:30]

array(['SAMSUNG SM-G892A Build/NRD90M', 'iOS Device', 'Windows', nan,
       'MacOS', 'SM-G930V Build/NRD90M', 'BLADE A602 Build/MRA58K',
       'XT1635-02 Build/NPN26.118-22-2', 'Z970', 'SM-N920V Build/NRD90M',
       'Redmi Note 4 Build/MMB29M', 'Lenovo PB1-750M Build/S100',
       'LT22i Build/6.2.A.1.100', 'rv:52.0', 'SM-G950U Build/NRD90M',
       'LG-H872 Build/NRD90U', 'LG-K500 Build/MMB29M',
       'SM-P550 Build/MMB29M', 'SM-J700M Build/MMB29K', 'Trident/7.0',
       'rv:57.0', 'SAMSUNG SM-G930T Build/NRD90M',
       'Blade V6 Plus Build/MRA58K', 'BLL-L23 Build/HUAWEIBLL-L23',
       'KYOCERA-C6742A Build/LMY47V', 'F3113 Build/33.2.A.4.70',
       'D5306 Build/19.4.A.0.182', 'M4 SS4457 Build/MRA58K',
       'SM-G955U Build/NRD90M', 'SM-G610M Build/MMB29K'], dtype=object)

In [54]:
train_identity.loc[:,'id_10'].mean()

-0.3011237754584523

In [68]:
fraud_df=train_transaction[lambda df: df.isFraud == True]

In [69]:
fraud_df.shape

(20663, 393)

In [72]:
id_ids=train_identity.index.tolist()

In [106]:
fraud_with_ids=fraud_df[lambda df: df.index.isin(id_ids)]

In [107]:
fraud_with_ids.shape

(11318, 393)